In [2]:
# -*- coding: utf-8 -*-
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import re
from string import digits
import nltk; nltk.download('stopwords')
import os
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
import pandas as pd
hotel_review = pd.read_csv('Hotel_Reviews.csv')
hotel_review.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968


In [4]:
# 나라 중복값 확인
w_count  = {}
for lst in hotel_review['Reviewer_Nationality']:
    try: w_count[lst] += 1
    except: w_count[lst]=1
print(w_count)

{' Russia ': 3900, ' Ireland ': 14827, ' Australia ': 21686, ' United Kingdom ': 245246, ' New Zealand ': 3237, ' Poland ': 2891, ' Belgium ': 6031, ' Norway ': 2383, ' France ': 7296, ' Italy ': 6114, ' Canada ': 7894, ' Netherlands ': 8772, ' Germany ': 7941, ' United States of America ': 35437, ' Hungary ': 2185, ' Israel ': 6610, ' Switzerland ': 8678, ' India ': 3437, ' United Arab Emirates ': 10235, ' Malaysia ': 1883, ' Denmark ': 1724, ' South Africa ': 3821, ' Rwanda ': 13, ' Finland ': 1829, ' Portugal ': 1839, ' Luxembourg ': 1359, ' Spain ': 4737, ' Malta ': 1685, ' China ': 3400, ' Turkey ': 5444, ' Ukraine ': 1102, ' Gibraltar ': 380, ' Argentina ': 517, ' Abkhazia Georgia ': 153, ' Isle of Man ': 405, ' Sweden ': 3368, ' Croatia ': 1340, ' Saudi Arabia ': 8951, ' Lithuania ': 727, ' Slovenia ': 796, ' Panama ': 122, ' Austria ': 2412, ' Brazil ': 1865, ' Taiwan ': 923, ' Iceland ': 933, ' Iran ': 1086, ' Liechtenstein ': 20, ' Chile ': 334, ' Hong Kong ': 3013, ' United 

In [5]:
# 대륙별 행 추출
hotel_review_Africa = hotel_review[['Reviewer_Nationality',"Negative_Review","Positive_Review"]]
Is = hotel_review_Africa[hotel_review_Africa['Reviewer_Nationality']  == " Israel "]
Uae = hotel_review_Africa[hotel_review_Africa['Reviewer_Nationality']  == " United Arab Emirates "]
Sa = hotel_review_Africa[hotel_review_Africa['Reviewer_Nationality']  == " South Africa "]
Sd = hotel_review_Africa[hotel_review_Africa['Reviewer_Nationality']  == " Saudi Arabia "]
Ba = hotel_review_Africa[hotel_review_Africa['Reviewer_Nationality']  == " Bahrain "]
Ku = hotel_review_Africa[hotel_review_Africa['Reviewer_Nationality']  == " Kuwait "]
AFRICA = pd.concat([Is,Uae,Sa,Sd,Ba,Ku])
AFRICA

,Reviewer_Nationality,Negative_Review,Positive_Review
35,Israel,Construction on site but not mentioned on boo...,Very nice hotel manager he upgraded us becaus...
184,Israel,The location unfortunately were a bit far fro...,The room is beautiful and comfortable the sho...
190,Israel,In our own experience First night we couldn t...,Staff was vey pleasant The place is very well...
205,Israel,At first glance it looks like a dream hotel B...,The best thing at this price excellent price ...
216,Israel,Its hard to get back to the hotel later at ni...,The hotel is beautiful the pictures here aren...
393,Israel,Long walk from the city center,No Positive
450,Israel,No Negative,Excellent location near major railway lines E...
612,Israel,No Negative,Often the reality is different from the websi...
629,Israel,The hotel is nice but the facilities are poor...,The hotel is clean in good location the break...
638,Israel,a bit petty but important extra socket near t...,very nice staff at the front desk


In [0]:
Negative = AFRICA.iloc[:,1]
Positive = AFRICA.iloc[:,2]

In [7]:
Negative = [re.sub("\'", "", str(sent)) for sent in Negative]
Negative = [re.sub("No Negative'", "", str(sent)) for sent in Negative]
pprint(Negative[:1])

[' Construction on site but not mentioned on booking com The breakfast can be '
 'improved it is quite expensive for its quality ']


In [8]:
Positive = [re.sub("\'", "", str(sent1)) for sent1 in Positive]
Positive = [re.sub("No Positive'", "", str(sent1)) for sent1 in Positive]
pprint(Positive[:1])

[' Very nice hotel manager he upgraded us because we had troubles with two '
 'different rooms They do everything to make you feel welcome and comfortable '
 'The park directly in front of the hotel is amazing rent a bike directly and '
 'make a tour Also the bathtub and bed is great ']


In [0]:
import gensim 
from gensim.models import Word2Vec

In [10]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words1 = list(sent_to_words(Negative))
data_words2 = list(sent_to_words(Positive))

print(data_words1[:1])
print(data_words2[:1])

[['construction', 'on', 'site', 'but', 'not', 'mentioned', 'on', 'booking', 'com', 'the', 'breakfast', 'can', 'be', 'improved', 'it', 'is', 'quite', 'expensive', 'for', 'its', 'quality']]
[['very', 'nice', 'hotel', 'manager', 'he', 'upgraded', 'us', 'because', 'we', 'had', 'troubles', 'with', 'two', 'different', 'rooms', 'they', 'do', 'everything', 'to', 'make', 'you', 'feel', 'welcome', 'and', 'comfortable', 'the', 'park', 'directly', 'in', 'front', 'of', 'the', 'hotel', 'is', 'amazing', 'rent', 'bike', 'directly', 'and', 'make', 'tour', 'also', 'the', 'bathtub', 'and', 'bed', 'is', 'great']]


In [11]:
%%time
model1 = Word2Vec(data_words1)

CPU times: user 5.49 s, sys: 32 ms, total: 5.52 s
Wall time: 2.97 s


In [12]:
%%time
model2 = Word2Vec(data_words2)

CPU times: user 4.24 s, sys: 29 ms, total: 4.27 s
Wall time: 2.33 s


In [13]:
model1.wv.most_similar("service")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('services', 0.6896906495094299),
 ('cleaning', 0.6695860028266907),
 ('wifi', 0.6527811884880066),
 ('dining', 0.6394953727722168),
 ('dinner', 0.6220106482505798),
 ('internet', 0.6212985515594482),
 ('food', 0.6207149028778076),
 ('restaurant', 0.611198365688324),
 ('attitude', 0.5833016633987427),
 ('breakfast', 0.563032329082489)]

In [14]:
model1.wv.most_similar("room")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('bed', 0.6607218980789185),
 ('suite', 0.6495183706283569),
 ('rooms', 0.6009031534194946),
 ('bathroom', 0.5961786508560181),
 ('bedroom', 0.5552880764007568),
 ('toom', 0.5528099536895752),
 ('glazing', 0.5360465049743652),
 ('beds', 0.5299645662307739),
 ('liking', 0.5155165195465088),
 ('king', 0.5049129128456116)]

In [15]:
model1.wv.most_similar("location")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('center', 0.7931280136108398),
 ('everything', 0.7906079292297363),
 ('far', 0.7672236561775208),
 ('city', 0.7572845220565796),
 ('central', 0.7569197416305542),
 ('its', 0.7452247738838196),
 ('best', 0.7434491515159607),
 ('london', 0.7249488234519958),
 ('convenient', 0.7215392589569092),
 ('heart', 0.7208604216575623)]

In [16]:
model1.wv.most_similar("transport")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('transportation', 0.9571740627288818),
 ('shopping', 0.9533759355545044),
 ('duomo', 0.948486328125),
 ('nearest', 0.9470086693763733),
 ('public', 0.9434255957603455),
 ('ride', 0.9426274299621582),
 ('tube', 0.9394738674163818),
 ('dam', 0.934174120426178),
 ('lines', 0.9275264143943787),
 ('km', 0.9252655506134033)]

In [17]:
model2.wv.most_similar("service")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('services', 0.7989984750747681),
 ('attitude', 0.7117817997932434),
 ('hospitality', 0.6902007460594177),
 ('cleaning', 0.6698524951934814),
 ('team', 0.6518614292144775),
 ('food', 0.6441803574562073),
 ('outstanding', 0.641688883304596),
 ('concierge', 0.631514310836792),
 ('housekeeping', 0.6273354887962341),
 ('especially', 0.6207024455070496)]

In [18]:
model2.wv.most_similar("room")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('bathroom', 0.7714527249336243),
 ('suite', 0.7706912755966187),
 ('rooms', 0.6962864995002747),
 ('apartment', 0.6811289191246033),
 ('king', 0.6761717796325684),
 ('shower', 0.6708083152770996),
 ('bath', 0.6599761843681335),
 ('junior', 0.6343474984169006),
 ('bed', 0.632392942905426),
 ('bedroom', 0.6240509152412415)]

In [19]:
model2.wv.most_similar("location")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('position', 0.6964618563652039),
 ('place', 0.650229275226593),
 ('everything', 0.5882082581520081),
 ('property', 0.5594277381896973),
 ('greenwich', 0.5542212128639221),
 ('elegance', 0.5507317781448364),
 ('neighborhood', 0.5421019792556763),
 ('leidseplein', 0.5398176312446594),
 ('spot', 0.5375691056251526),
 ('environment', 0.5369924306869507)]

In [20]:
model2.wv.most_similar("transport")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('public', 0.9748728275299072),
 ('transportation', 0.9729366898536682),
 ('stations', 0.9133415818214417),
 ('buses', 0.907095730304718),
 ('tourist', 0.9069772362709045),
 ('museums', 0.9060170650482178),
 ('trains', 0.9051945209503174),
 ('accessible', 0.9036531448364258),
 ('subway', 0.8965941071510315),
 ('mall', 0.8907855153083801)]